In [72]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
chRNA_psi = pd.read_csv('../code/QTLs/QTLTools/polyA.Splicing/OnlyFirstReps.PSI.bed.gz', sep='\t')

In [ ]:
14829

In [11]:
for x in chRNA_psi.groupby('gid'):
    if x[1].shape[0] == 3:
        break

In [ ]:
# def get_cassette_exon(clu_df):
#     for idx, row in clu_df

In [12]:
x[1]

,#Chr,start,end,pid,gid,strand,HG00096,HG00097,HG00099,HG00100,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
20127,chr10,180128,197911,10:180128:197911:clu_24834_+,chr10_clu_24834_+,+,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
20128,chr10,180128,209889,10:180128:209889:clu_24834_+,chr10_clu_24834_+,+,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.971430,1.0,1.0,1.0,1.0
20129,chr10,198005,209889,10:198005:209889:clu_24834_+,chr10_clu_24834_+,+,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0


In [17]:
x[1].loc[(x[1].start == x[1].start.min()) & (x[1].end == x[1].end.max())]

,#Chr,start,end,pid,gid,strand,HG00096,HG00097,HG00099,HG00100,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
20128,chr10,180128,209889,10:180128:209889:clu_24834_+,chr10_clu_24834_+,+,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.97143,1.0,1.0,1.0,1.0


In [23]:
((x[1].start == x[1].start.min()) & (x[1].end == x[1].end.max())).sum()

1

In [20]:
(x[1].start == x[1].start.min()).sum()

2

In [199]:
def is_cassette(df):
    
    cassette_list = []
    
    samples = df.columns[6:]
    
    any_cassette = False
    
    for idx, row in df.iterrows():
        
        if (row[samples].isna().mean()) < 0.9:
        
            start = row.start
            end = row.end

            if np.sum(df.start == start) >= 2:
                if np.sum(df.end == end) >= 2:
                    if np.sum((df.start == start) & (df.end <end).sum()>=1):
                        if np.sum((df.start > start) & (df.end == end).sum()>=1):
                            I1 = df.loc[(df.start == start) & (df.end < end)]
                            I2 = df.loc[(df.start > start) & (df.end == end)]

                            for idx_i1, row_i1 in I1.iterrows():
                                for idx_i2, row_i2 in I2.iterrows():
                                    if int(row_i1.end) < (int(row_i2.start)-2):
                                        
                                        length = int(row_i2.start) - int(row_i1.end) - 2
                                        
                                        cassette = (row.pid, row_i1.pid, row_i2.pid)
                                        PSI = np.mean(row_i1[samples] + row_i2[samples])
                                        
                                        cassette_list.append((cassette, PSI, length))
                                        
                                        any_cassette =  True

    return cassette_list
                    
    

In [ ]:
cassette_list = []

for x in tqdm(chRNA_psi.groupby('gid')):
    cassette_list.extend(is_cassette(x[1]))


 13%|███████████████████▋                                                                                                                                | 6556/49245 [00:37<04:33, 156.09it/s]

In [ ]:
bins_1 = []
bins_2 = []
bins_3 = []
bins_4 = []
bins_5 = []
for exon in cassette_list:
#     print(exon[1])
#     print(exon[2])
    if (exon[1] < 0.2):
#         print('1')
        bins_1.append(exon[2]%3==0)
    elif (exon[1] < 0.4):
        bins_2.append(exon[2]%3==0)
    elif (exon[1] < 0.6):
        bins_3.append(exon[2]%3==0)
    elif (exon[1] < 0.8):
        bins_4.append(exon[2]%3==0)
    else:
        bins_5.append(exon[2]%3==0)

In [ ]:
np.mean(bins_1)

In [166]:
bins_pct = [np.mean(x) for x in bins]

0.3786830108459868

In [60]:
I1 = x[1].loc[(x[1].start == x[1].start.min()) & (x[1].end < x[1].end.max())]
I2 = x[1].loc[(x[1].start > x[1].start.min()) & (x[1].end == x[1].end.max())]
SE = x[1].loc[(x[1].start == x[1].start.min()) & (x[1].end == x[1].end.max())]

length_SE = int(SE.end) - int(SE.start)
exon_length = I2.start - I1.end
symmetric = (exon_length%3 == 0)

avg_PSI = np.nanmean(I1[x[1].columns[6:]] + I2[x[1].columns[6:]], axis=1)


/tmp/ipykernel_715461/1515007243.py:9: RuntimeWarning: Mean of empty slice
  avg_PSI = np.nanmean(I1[x[1].columns[6:]] + I2[x[1].columns[6:]], axis=1)


In [65]:
x[1]

,#Chr,start,end,pid,gid,strand,HG00096,HG00097,HG00099,HG00100,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
22123,chr10,38017015,38017291,10:38017015:38017291:clu_25065_+,chr10_clu_25065_+,+,0.923080,0.8913,0.970590,0.958330,...,0.903230,0.9375,0.87879,0.926830,0.83333,0.945950,0.941180,0.913040,0.935480,0.92
22124,chr10,38017015,38017312,10:38017015:38017312:clu_25065_+,chr10_clu_25065_+,+,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
22125,chr10,38017020,38017291,10:38017020:38017291:clu_25065_+,chr10_clu_25065_+,+,0.076923,0.1087,0.029412,0.041667,...,0.096774,0.0625,0.12121,0.073171,0.16667,0.054054,0.058824,0.086957,0.064516,0.08


In [68]:
((x[1].start == x[1].start.min()) & (x[1].end == x[1].end.max())).sum()

1